### The best model for predicting iris species from the iris dataset according to its accuracy

# imports

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from matplotlib.colors import ListedColormap
import numpy as np